In [1]:
from refexps import load_data, move_level, gamerun_level, get_previous #, lex_sim
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats
import compound_splitter
from scipy import stats
from nltk import word_tokenize
from string import punctuation
punct = list(punctuation)

In [5]:
def exp_len(exp, language, splits, compound):
    '''
    Extract content words by the phrase
    :param exp: referring expression tuple
    :return: set of content words of the phrase
    '''
    exp = exp[2]
    words = word_tokenize(exp, language)
    if compound == True:
        if language == 'german':
            words_nocomp = []
            for w in words:
                word = compound_splitter.split_word(w, splits)
                words_nocomp = words_nocomp + (word.split(' '))	
                words = words_nocomp
    return len(words)

def len_div(exp1, previous, language, splits, compound):
    '''
    compute dice similarity between the content words in two referring expression
    :param exp1: referring expression tuple 1
    :param exp2: referring expression tuple 2
    :return: dice similarity between the content words in the expressions; undefined if they have none
    '''        
    len_exp = exp_len(exp1, language, splits, compound)    
    previous_len = np.mean(previous)
    metric = float(previous_len - len_exp)/float(previous_len + len_exp)
    return metric if not np.isnan(metric) and metric > -1  and metric < 1 else 'undefined'

In [ ]:
def compute_values_move(refexps_dir, success_dir, compound=True):
    splits = compound_splitter.load_dict('de_lower.dict')
    move_alignment = defaultdict(list)

    for dialogue in listdir(refexps_dir):
        if dialogue.startswith('FTT'):
            language = 'german'
        else:
            language = 'english'
        dialogue = path.join(refexps_dir, dialogue)
        #load referring expressions data
        dialogue = load_data(dialogue)
        #move level
        dialogue_move = move_level(dialogue)
        #iterate over moves
        for move in dialogue_move:
            refs = move[1]
            #iterate over objects
            for obj in refs:
                #iterate over objects
                refobj = refs[obj]
                #SIMILARITY OF LEXICAL CONTENT INTO
                prev_length = []
                speakers = set()
                divs = []
                divs_speakers = defaultdict(list)
                for ref in refobj:
                    speaker = ref[1]
                    speakers.add(speaker)
                    if prev_length != []:
                        div = len_div(ref, prev_length, language, splits, compound)
                        if div != 'undefined':
                            divs.append(div)
                            divs_speakers[speaker].append(div)
                    len_exp = exp_len(ref, language, splits, compound) 
                    prev_length.append(len_exp)
                if divs != []:
                    alignment = np.mean(divs)
                    move_alignment['Alignment'].append(alignment)
                for s in speakers:
                    if divs_speakers[speaker] != []:
                        alignment = np.mean(divs_speakers[speaker])
                        move_alignment['Alignment '+s].append(alignment)
    for value in move_alignment:
        dataset = move_alignment[value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tStandard deviation: '+ str(np.std(dataset))
        print '\tMax value: ' + str(max(dataset)) + '\tMin value: '+ str(min(dataset))
    return move_alignment

In [11]:
de_dataset_dir = 'En_De_Dataset/De/RefExp'
de_success_dir = 'En_De_Dataset/De/Success'

en_dataset_dir = 'En_De_Dataset/En/RefExp'
en_success_dir = 'En_De_Dataset/En/Success'

all_dataset_dir = 'En_De_Dataset/All/RefExp'
all_success_dir = 'En_De_Dataset/All/Success'

results_de = compute_values_move(de_dataset_dir, de_success_dir)
print '\n\n'
results_en = compute_values_move(en_dataset_dir, en_success_dir)
print '\n\n'
results_all = compute_values_move(all_dataset_dir, all_success_dir)

	Alignment p-utts: 
	Mean: 0.125334379461	Standard deviation: 0.274491868877
	Max value: 0.857142857143	Min value: -0.714285714286
	Alignment: 
	Mean: 0.119894400262	Standard deviation: 0.272796712862
	Max value: 0.857142857143	Min value: -0.714285714286
	Alignment e-utts: 
	Mean: 0.136266563549	Standard deviation: 0.27681838222
	Max value: 0.857142857143	Min value: -0.714285714286



	Alignment p-utts: 
	Mean: 0.110288887939	Standard deviation: 0.229964740594
	Max value: 0.846153846154	Min value: -0.571428571429
	Alignment: 
	Mean: 0.0952154319612	Standard deviation: 0.230235031267
	Max value: 0.846153846154	Min value: -0.571428571429
	Alignment e-utts: 
	Mean: 0.105525507044	Standard deviation: 0.243777371754
	Max value: 0.634920634921	Min value: -0.571428571429



	Alignment p-utts: 
	Mean: 0.120511736305	Standard deviation: 0.261142057483
	Max value: 0.857142857143	Min value: -0.714285714286
	Alignment: 
	Mean: 0.111712305036	Standard deviation: 0.259720559155
	Max value: 0.8571428

In [ ]:
results_de_nocs = compute_values_move(de_dataset_dir, de_success_dir, compound = False)
print '\n\n'

for var in results_de:
    mannwhitneyu =  stats.mannwhitneyu(results_de[var],results_de_nocs[var], alternative='two-sided')
    if mannwhitneyu[1] < 0.05:
        print var
        print str(mannwhitneyu)+'\tSignificantly different'
        print np.mean(results_de[var]), np.mean(results_de_nocs[var])

In [12]:
print 'Differences between languages'
for var in results_all:
    mannwhitneyu =  stats.mannwhitneyu(results_de[var],results_en[var], alternative='two-sided')
    if mannwhitneyu[1] < 0.05:
        print var
        print str(mannwhitneyu)+'\tSignificantly different'
print '\n'

print 'Differences between players'
mannwhitneyu =  stats.mannwhitneyu(results_de['Alignment e-utts'],results_de['Alignment p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move alignment de'
    print str(mannwhitneyu)+'\tSignificantly different'

mannwhitneyu =  stats.mannwhitneyu(results_en['Alignment e-utts'],results_en['Alignment p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move alignment en'
    print str(mannwhitneyu)+'\tSignificantly different'
                                                                        
mannwhitneyu =  stats.mannwhitneyu(results_all['Alignment e-utts'],results_all['Alignment p-utts'], alternative='two-sided')
if mannwhitneyu[1] < 0.05:
    print 'Players move alignment all'
    print str(mannwhitneyu)+'\tSignificantly different'                                                             
                                                                    

Differences between languages


Differences between players


In [6]:
def compute_values_game(refexps_dir, success_dir, compound = True):
    splits = compound_splitter.load_dict('de_lower.dict')
    game_alignment = defaultdict(list)
    for dialogue in listdir(refexps_dir):
        if dialogue.startswith('FTT'):
            language = 'german'
        else:
            language = 'english'
        dialogue = load_data(path.join(refexps_dir, dialogue))
        dialogue_game = gamerun_level(dialogue)
        for obj in dialogue_game:
            refobj = dialogue_game[obj]
            prev_length = []
            speakers = set()
            divs = []
            divs_speakers = defaultdict(list)
            for ref in refobj:
                speaker = ref[1]
                speakers.add(speaker)
                if prev_length != []:
                    div = len_div(ref, prev_length, language, splits, compound)
                    if div != 'undefined':
                        divs.append(div)
                        divs_speakers[speaker].append(div)
                len_exp = exp_len(ref, language, splits, compound) 
                prev_length.append(len_exp)
            if divs != []:
                alignment = np.mean(divs)
                game_alignment['Alignment'].append(alignment)
            for s in speakers:
                if divs_speakers[speaker] != []:
                    alignment = np.mean(divs_speakers[speaker])
                    game_alignment['Alignment '+s].append(alignment)
    for value in game_alignment:
        dataset = game_alignment[value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tStandard deviation: '+ str(np.std(dataset))
        print '\tMax value: ' + str(max(dataset)) + '\tMin value: '+ str(min(dataset))
    return game_alignment


In [7]:
de_dataset_dir = 'En_De_Dataset/De/RefExp'
de_success_dir = 'En_De_Dataset/De/Success'

en_dataset_dir = 'En_De_Dataset/En/RefExp'
en_success_dir = 'En_De_Dataset/En/Success'

all_dataset_dir = 'En_De_Dataset/All/RefExp'
all_success_dir = 'En_De_Dataset/All/Success'

results_de = compute_values_game(de_dataset_dir, de_success_dir)
print '\n\n'
results_en = compute_values_game(en_dataset_dir, en_success_dir)
print '\n\n'
results_all = compute_values_game(all_dataset_dir, all_success_dir)

	Alignment p-utts: 
	Mean: 0.110110297433	Standard deviation: 0.237414972105
	Max value: 0.75	Min value: -0.714285714286
	Alignment: 
	Mean: 0.110839287544	Standard deviation: 0.2268166911
	Max value: 0.75	Min value: -0.714285714286
	Alignment e-utts: 
	Mean: 0.102930478603	Standard deviation: 0.208908597768
	Max value: 0.714285714286	Min value: -0.714285714286



	Alignment p-utts: 
	Mean: 0.0603350300085	Standard deviation: 0.220690275817
	Max value: 0.647864906832	Min value: -0.818181818182
	Alignment: 
	Mean: 0.0785209763695	Standard deviation: 0.181818114489
	Max value: 0.647864906832	Min value: -0.571428571429
	Alignment e-utts: 
	Mean: 0.0393684392322	Standard deviation: 0.240234900116
	Max value: 0.647864906832	Min value: -0.818181818182



	Alignment p-utts: 
	Mean: 0.090713631617	Standard deviation: 0.232313313592
	Max value: 0.75	Min value: -0.818181818182
	Alignment: 
	Mean: 0.0978221899878	Standard deviation: 0.21045376111
	Max value: 0.75	Min value: -0.714285714286
	Align

In [16]:
results_de_nocs = compute_values_game(de_dataset_dir, de_success_dir, compound = False)
print '\n\n'

for var in results_de:
    mannwhitneyu =  stats.mannwhitneyu(results_de[var],results_de_nocs[var], alternative='two-sided')
    if mannwhitneyu[1] < 0.05:
        print var
        print str(mannwhitneyu)+'\tSignificantly different'
        print np.mean(results_de[var]), np.mean(results_de_nocs[var])

	Alignment p-utts: 
	Mean: 0.110771195057	Standard deviation: 0.235296558722
	Max value: 0.75	Min value: -0.714285714286
	Alignment: 
	Mean: 0.111264108606	Standard deviation: 0.225790136582
	Max value: 0.75	Min value: -0.714285714286
	Alignment e-utts: 
	Mean: 0.101420266777	Standard deviation: 0.204440782441
	Max value: 0.714285714286	Min value: -0.714285714286





In [8]:
print 'Differences between languages'
for var in results_all:
    ttest =  stats.mannwhitneyu(results_de[var],results_en[var], alternative='two-sided')
    if ttest[1] < 0.05:
        print var
        print str(ttest)+'\tSignificantly different'
print '\n'

print 'Differences between players'
ttest =  stats.mannwhitneyu(results_de['Alignment e-utts'],results_de['Alignment p-utts'], alternative='two-sided')
if ttest[1] < 0.05:
    print 'Players game alignment de'
    print str(ttest)+'\tSignificantly different'

ttest =  stats.mannwhitneyu(results_en['Alignment e-utts'],results_en['Alignment p-utts'], alternative='two-sided')
if ttest[1] < 0.05:
    print 'Players game alignment en'
    print str(ttest)+'\tSignificantly different'
                                                                        
ttest =  stats.mannwhitneyu(results_all['Alignment e-utts'],results_all['Alignment p-utts'], alternative='two-sided')
if ttest[1] < 0.05:
    print 'Players game alignment all'
    print str(ttest)+'\tSignificantly different'                                                               

Differences between languages
Alignment p-utts
MannwhitneyuResult(statistic=16838.5, pvalue=0.01043326118246455)	Significantly different
Alignment
MannwhitneyuResult(statistic=17688.5, pvalue=0.030074076978341489)	Significantly different
Alignment e-utts
MannwhitneyuResult(statistic=6783.5, pvalue=0.033790474614030862)	Significantly different


Differences between players
